In [ ]:
!python filter_srr.py "SRR5090597_putative_insertion_table.csv" "SRR5090597" "hpv16" "UCSChg38" 

In [ ]:
!pip install pandas

In [ ]:
!sudo apt-get -y update
!sudo apt-get -y install autoconf automake make gcc perl zlib1g-dev libbz2-dev liblzma-dev libcurl4-gnutls-dev libssl-dev libncurses5-dev
!wget https://github.com/samtools/samtools/releases/download/1.14/samtools-1.14.tar.bz2
!tar xvjf samtools-1.14.tar.bz2
!cd samtools-1.14
!make
!export PATH=$PATH:`pwd`

In [59]:
import igv
import time
import pandas
import subprocess

In [48]:
insertion_table_name = 'test_table.csv'
srr = "SRR5090597"
srr_id = 1
donor_name = "hpv16"
recipient_name = "UCSChg38"
path = "../../wallaby/workflows/outputs/hpv16_rnaseq/"

In [49]:
insertion_table = pandas.read_csv(insertion_table_name)
insertion_table

,srr,id,chr,start,stop,num_crossings,unique_crossings,num_reads,gene_name
0,SRR5090597,1,chr1,150576896,150577494,3,1Md_2Mr|1Md_1Mr|2Md_1Mr,386,MCL1
1,SRR5090597,2,chr1,150578243,150578495,3,1Md_2Mr|1Md_1Mr|2Md_1Mr,528,MCL1


In [50]:
crossings = ["1Md_2Mr", "1Md_1Mr", "2Md_1Mr"]
padding_left = 0
padding_right = 0

recipient_1Md_2Mr = path + srr + "-to-UCSChg38_" + "1Md_2Mr" + "_filtered.bam"
recipient_1Md_1Mr = path + srr + "-to-UCSChg38_" + "1Md_1Mr" + "_filtered.bam"
recipient_2Md_1Mr = path + srr + "-to-UCSChg38_" + "2Md_1Mr" + "_filtered.bam"

donor_1Md_2Mr = path + srr + "-to-hpv16_" + "1Md_2Mr" + "_filtered.bam"
donor_1Md_1Mr = path + srr + "-to-hpv16_" + "1Md_1Mr" + "_filtered.bam"
donor_2Md_1Mr = path + srr + "-to-hpv16_" + "2Md_1Mr" + "_filtered.bam"

In [60]:
for srr_id in insertion_table["id"]:
    insertion_table_srr = insertion_table[insertion_table["srr"] == srr]
    insertion_table_srr_id = insertion_table_srr[insertion_table_srr["id"] == srr_id]
    chromosome = str(insertion_table_srr_id['chr'].values[0])
    start = str(insertion_table_srr_id['start'].values[0] - padding_left)
    stop = str(insertion_table_srr_id['stop'].values[0] + padding_right)
    locus = str(chromosome + ":" + start + "-" + stop)
    
    subprocess.getoutput("/tmp/samtools-1.14/samtools view " + recipient_1Md_2Mr + " " + locus + " | cut -f1 | sort | uniq > reads1.txt")
    subprocess.getoutput("/tmp/samtools-1.14/samtools view " + recipient_1Md_1Mr + " " + locus + " | cut -f1 | sort | uniq > reads2.txt")
    subprocess.getoutput("/tmp/samtools-1.14/samtools view " + recipient_2Md_1Mr + " " + locus + " | cut -f1 | sort | uniq > reads3.txt")
    subprocess.getoutput("cat reads1.txt reads2.txt reads3.txt > reads.txt")
    subprocess.getoutput("rm reads1.txt reads2.txt reads3.txt")

    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + donor_name + "_1Md_2Mr_filtered_" + str(srr_id) + ".bam " + donor_1Md_2Mr)
    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + donor_name + "_1Md_1Mr_filtered_" + str(srr_id) + ".bam " + donor_1Md_1Mr)
    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + donor_name + "_2Md_1Mr_filtered_" + str(srr_id) + ".bam " + donor_2Md_1Mr)
    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + recipient_name + "_1Md_2Mr_filtered_" + str(srr_id) + ".bam " + recipient_1Md_2Mr)
    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + recipient_name + "_1Md_1Mr_filtered_" + str(srr_id) + ".bam " + recipient_1Md_1Mr)
    subprocess.getoutput("/tmp/samtools-1.14/samtools view -N reads.txt -o " + srr + "-to-" + recipient_name + "_2Md_1Mr_filtered_" + str(srr_id) + ".bam " + recipient_2Md_1Mr)